# Algorithm 1

In [1]:
import networkx as nx
import glob, os
import numpy as np
from tqdm.notebook import tqdm
from itertools import islice
import statistics
from numpy.random import choice
import math

In [2]:
def nodes_connected(G, u, v):
  return u in G.neighbors(v)

In [3]:
def s(id):
  if int(id[0]) <= int(id[1]): return id
  else: return id[1], id[0]

In [4]:
def sort_dict_by_value(d, reverse = False):
  return dict(sorted(d.items(), key = lambda x: x[1], reverse = reverse))

# Train oracle for oregon dataset

In [6]:
data_list = []
with open("/content/oregon/oregon_1.txt") as f:
  lines = f.readlines()[4:]
  data_list.append(lines)

with open("/content/oregon/oregon_4.txt") as f:
  lines = f.readlines()[4:]
  data_list.append(lines)


train_list = data_list[0]
test_list = data_list[1]


G_train = nx.Graph()
for el in train_list:
    values = el.strip().split()
    G_train.add_edges_from([(values[0],values[1])])


G_test = nx.Graph()
for el in test_list:
    values = el.strip().split()
    G_test.add_edges_from([(values[0],values[1])])


score_matrix = dict()
for idx,el in enumerate(list(G_train.edges())):
  u = el[0]
  v = el[1]  
  u_neigh = set(list(G_train.neighbors(u)))
  v_neigh = set(list(G_train.neighbors(v)))
  common_neighbour = set.intersection(u_neigh,v_neigh) 
  N_uv = len(common_neighbour)
  score_matrix[s((u,v))] = N_uv



score_matrix_sorted = sort_dict_by_value(score_matrix, True)

oracle = dict()
for key, value in islice(score_matrix_sorted.items(), 0, int(len(score_matrix_sorted)/10)):
  oracle.update({s(key):value})

# Train oracle for caida 2006 dataset

In [5]:
caida_2006_data_list = []
with open("/content/caida_2006/caida_1.txt") as f:
  lines = f.readlines()[8:]
  caida_2006_data_list.append(lines)

with open("/content/caida_2006/caida_30.txt") as f:
  lines = f.readlines()[8:]
  caida_2006_data_list.append(lines)

caida_2006_train_list = caida_2006_data_list[0]
caida_2006_test_list = caida_2006_data_list[1]


caida_2006_G_train = nx.Graph()
for el in caida_2006_train_list:
    values = el.strip().split()
    caida_2006_G_train.add_edges_from([(values[0],values[1])])


caida_2006_G_test = nx.Graph()
for el in caida_2006_test_list:
    values = el.strip().split()
    caida_2006_G_test.add_edges_from([(values[0],values[1])])


caida_2006_score_matrix = dict()
for idx,el in enumerate(list(caida_2006_G_train.edges())):
  u = el[0]
  v = el[1]  
  u_neigh = set(list(caida_2006_G_train.neighbors(u)))
  v_neigh = set(list(caida_2006_G_train.neighbors(v)))
  common_neighbour = set.intersection(u_neigh,v_neigh) 
  N_uv = len(common_neighbour)
  caida_2006_score_matrix[s((u,v))] = N_uv



caida_2006_score_matrix_sorted = sort_dict_by_value(caida_2006_score_matrix, True)

caida_2006_oracle = dict()
for key, value in islice(caida_2006_score_matrix_sorted.items(), 0, int(len(caida_2006_score_matrix_sorted)/10)):
  caida_2006_oracle.update({s(key):value})

# Train oracle for caida 2007 dataset

In [5]:
caida_2007_data_list = []
with open("/content/caida_2007/caida_1.txt") as f:
  lines = f.readlines()[8:]
  caida_2007_data_list.append(lines)

with open("/content/caida_2007/caida_25.txt") as f:
  lines = f.readlines()[8:]
  caida_2007_data_list.append(lines)

caida_2007_train_list = caida_2007_data_list[0]
caida_2007_test_list = caida_2007_data_list[1]


caida_2007_G_train = nx.Graph()
for el in caida_2007_train_list:
    values = el.strip().split()
    caida_2007_G_train.add_edges_from([(values[0],values[1])])


caida_2007_G_test = nx.Graph()
for el in caida_2007_test_list:
    values = el.strip().split()
    caida_2007_G_test.add_edges_from([(values[0],values[1])])


caida_2007_score_matrix = dict()
for idx,el in enumerate(list(caida_2007_G_train.edges())):
  u = el[0]
  v = el[1]  
  u_neigh = set(list(caida_2007_G_train.neighbors(u)))
  v_neigh = set(list(caida_2007_G_train.neighbors(v)))
  common_neighbour = set.intersection(u_neigh,v_neigh) 
  N_uv = len(common_neighbour)
  caida_2007_score_matrix[s((u,v))] = N_uv



caida_2007_score_matrix_sorted = sort_dict_by_value(caida_2007_score_matrix, True)

caida_2007_oracle = dict()
for key, value in islice(caida_2007_score_matrix_sorted.items(), 0, int(len(caida_2007_score_matrix_sorted)/10)):
  caida_2007_oracle.update({s(key):value})

# Test for oregon dataset

In [7]:
m = G_test.number_of_edges()
n = G_test.number_of_nodes()
T = sum(score_matrix.values())/3
A_l = 0
A_m = 0
A_h = 0
S_l = set()
S_m = set()
S_aux = set()
eps =  0.05
rho = 0
alpha = 0.085
beta = 0.6
gamma = 0.015
C = dict()
B = dict()
if T >= np.sqrt(m/eps) :
  rho = (m*T)**(1/3)
  p1 = (alpha*(eps**-2))/rho
  p2 = min((beta*(eps**-2)*rho)/T, 1)
  p3 = (gamma*(eps**-2)*np.log2(n))/rho
else:
  rho = ((m)**1/2)/eps
  p1 = 0
  p2 = 1
  p3 = (gamma*(eps**-2))/rho

sum_all = 0 

pbar = tqdm(G_test.nodes())
for v in pbar:
  for mem in S_l.union(S_m):
    if nodes_connected(G_test, str(mem[0]),v) and nodes_connected(G_test, str(mem[1]),v):
      C[s(mem)] += 1

  for mem in S_aux:
    if mem[0] == v or mem[1] == v:
        B[s(mem)] = 1
      
  for el in list(G_test.edges(v)):
    u = el[1]
    el = s(el)

    if np.random.uniform(0,1,1) <= p3:
      S_aux.add(el)
      B[el] = 0

    if el in oracle:
      edge_score = oracle[el]
    else:
      edge_score = 0

    if edge_score < T/rho: #this edge is light
      if el in S_l:
        A_l += C[el]
      else:
        if np.random.uniform(0,1,1) <= p1:
          S_l.add(el)
          C[el] = 0

    else:
      counter = 0
      for z in list(G_test.neighbors(u)):
        if nodes_connected(G_test, z,v) and s((u,z)) in S_aux:
            if  B[s((u,z))]==1:
              counter +=1

      if counter >= (p3*rho):
        A_h += counter
      elif el in S_m:
        A_m += C[el]
      else:
        if np.random.uniform(0,1,1) <= p2:
          S_m.add(el)
          C[el] = 0

temp1 = sum_all
temp2 = nx.triangles(G_test,v)
sum_all = int(A_l/p1 + A_m/p2 + A_h/p3)

 
print("predicted number of triangles: ", sum_all)
temp = sum(nx.triangles(G_test).values())/3
print("real number of triangles: ", temp)

  0%|          | 0/10859 [00:00<?, ?it/s]

predicted number of triangles:  18129
real number of triangles:  19108.0


In [7]:
m = caida_2006_G_test.number_of_edges()
n = caida_2006_G_test.number_of_nodes()
T = sum(caida_2006_score_matrix.values())/3
A_l = 0
A_m = 0
A_h = 0
S_l = set()
S_m = set()
S_aux = set()
eps =  0.05
rho = 0
alpha = 0.085
beta = 0.6
gamma = 0.015
C = dict()
B = dict()

if T >= np.sqrt(m/eps) :
  rho = (m*T)**(1/3)
  p1 = (alpha*(eps**-2))/rho
  p2 = min((beta*(eps**-2)*rho)/T, 1)
  p3 = (gamma*(eps**-2)*np.log2(n))/rho
else:
  rho = ((m)**1/2)/eps
  p1 = 0
  p2 = 1
  p3 = (gamma*(eps**-2))/rho

sum_all = 0 

pbar = tqdm(caida_2006_G_test.nodes())
for v in pbar:
  for mem in S_l.union(S_m):
    if nodes_connected(caida_2006_G_test, str(mem[0]),v) and nodes_connected(caida_2006_G_test, str(mem[1]),v):
      C[s(mem)] += 1

  for mem in S_aux:
    if mem[0] == v or mem[1] == v:
        B[s(mem)] = 1
      
  for el in list(caida_2006_G_test.edges(v)):
    u = el[1]
    el = s(el)

    if np.random.uniform(0,1,1) <= p3:
      S_aux.add(el)
      B[el] = 0

    if el in caida_2006_oracle:
      edge_score = caida_2006_oracle[el]
    else:
      edge_score = 0

    if edge_score < T/rho: #this edge is light
      if el in S_l:
        A_l += C[el]
      else:
        if np.random.uniform(0,1,1) <= p1:
          S_l.add(el)
          C[el] = 0

    else:
      counter = 0
      for z in list(caida_2006_G_test.neighbors(u)):
        if nodes_connected(caida_2006_G_test, z,v) and s((u,z)) in S_aux:
            if  B[s((u,z))]==1:
              counter +=1

      if counter >= (p3*rho):
        A_h += counter
      elif el in S_m:
        A_m += C[el]
      else:
        if np.random.uniform(0,1,1) <= p2:
          S_m.add(el)
          C[el] = 0


sum_all = int(A_l/p1 + A_m/p2 + A_h/p3)
  
print("predicted number of triangles: ", sum_all)
temp = sum(nx.triangles(caida_2006_G_test).values())/3
print("real number of triangles: ", temp)

  0%|          | 0/22602 [00:00<?, ?it/s]

predicted number of triangles:  37274
real number of triangles:  34926.0


In [8]:
m = caida_2007_G_test.number_of_edges()
n = caida_2007_G_test.number_of_nodes()
T = sum(caida_2007_score_matrix.values())/3
A_l = 0
A_m = 0
A_h = 0
S_l = set()
S_m = set()
S_aux = set()
eps =  0.05
rho = 0
alpha = 0.085
beta = 0.6
gamma = 0.015
C = dict()
B = dict()

if T >= np.sqrt(m/eps) :
  rho = (m*T)**(1/3)
  p1 = (alpha*(eps**-2))/rho
  p2 = min((beta*(eps**-2)*rho)/T, 1)
  p3 = (gamma*(eps**-2)*np.log2(n))/rho
else:
  rho = ((m)**1/2)/eps
  p1 = 0
  p2 = 1
  p3 = (gamma*(eps**-2))/rho

sum_all = 0 

pbar = tqdm(caida_2007_G_test.nodes())
for v in pbar:
  for mem in S_l.union(S_m):
    if nodes_connected(caida_2007_G_test, str(mem[0]),v) and nodes_connected(caida_2007_G_test, str(mem[1]),v):
      C[s(mem)] += 1

  for mem in S_aux:
    if mem[0] == v or mem[1] == v:
        B[s(mem)] = 1
      
  for el in list(caida_2007_G_test.edges(v)):
    u = el[1]
    el = s(el)

    if np.random.uniform(0,1,1) <= p3:
      S_aux.add(el)
      B[el] = 0

    if el in caida_2007_oracle:
      edge_score = caida_2007_oracle[el]
    else:
      edge_score = 0

    if edge_score < T/rho: #this edge is light
      if el in S_l:
        A_l += C[el]
      else:
        if np.random.uniform(0,1,1) <= p1:
          S_l.add(el)
          C[el] = 0

    else:
      counter = 0
      for z in list(caida_2007_G_test.neighbors(u)):
        if nodes_connected(caida_2007_G_test, z,v) and s((u,z)) in S_aux:
            if  B[s((u,z))]==1:
              counter +=1

      if counter >= (p3*rho):
        A_h += counter
      elif el in S_m:
        A_m += C[el]
      else:
        if np.random.uniform(0,1,1) <= p2:
          S_m.add(el)
          C[el] = 0

sum_all = int(A_l/p1 + A_m/p2 + A_h/p3)

print("predicted number of triangles: ", sum_all)
temp = sum(nx.triangles(caida_2007_G_test).values())/3
print("real number of triangles: ", temp)

  0%|          | 0/25304 [00:00<?, ?it/s]

predicted number of triangles:  37331
real number of triangles:  38812.0
